In [1]:
import logging
import time
import os
import pickle
import math

import matplotlib.pyplot as plt

#import tensorflow_datasets as tfds
import tensorflow as tf

# Import tf_text to load the ops used by the tokenizer saved model
#import tensorflow_text  # pylint: disable=unused-import
import pandas as pd
import numpy as np
import re
import seaborn as sns

from sklearn.model_selection import train_test_split

import jellyfish 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model,  Sequential
from tensorflow.keras.layers import LSTM, GRU, Bidirectional, Dropout, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding, Concatenate
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import Attention
from tensorflow.keras.optimizers import Adam, Adagrad
from keras.losses import sparse_categorical_crossentropy
logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings
import random

In [2]:
def AA_Codon_list():
    dic_AA_codon = {'A': ['GCT', 'GCC', 'GCA', 'GCG'],
                    'C': ['TGT', 'TGC'],
                    'D': ['GAT', 'GAC'],
                    'E': ['GAA', 'GAG'],
                    'F': ['TTT', 'TTC'],
                    'G': ['GGT', 'GGA', 'GGC', 'GGG'],
                    'H': ['CAT', 'CAC'],
                    'I': ['ATT', 'ATC', 'ATA'],
                    'K': ['AAA', 'AAG'],
                    'L': ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'],
                    'M': ['ATG'],
                    'N': ['AAT', 'AAC'],
                    'P': ['CCT', 'CCC', 'CCA', 'CCG'],
                    'Q': ['CAA', 'CAG'],
                    'R': ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'],
                    'S': ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'],
                    'T': ['ACT', 'ACC', 'ACA', 'ACG'],
                    'V': ['GTT', 'GTC', 'GTA', 'GTG'],
                    'W': ['TGG'],
                    'Y': ['TAT', 'TAC'],
                    '*': ['TAA', 'TAG', 'TGA']}

    codon_list = []
    AA_list = []
    for key in dic_AA_codon:
        for i in range(len(dic_AA_codon[key])):
            AA_list.append(key)
        for i in dic_AA_codon[key]:
            codon_list.append(i)
    return AA_list, codon_list

### GC content
### CSI
### %Rare codon

In [3]:
Data = {}
Prop = {}
Codon_Table = {}
Max_freq = {}

organism = ['Pichia', 'Human']
for o in organism:
    Codon_Table[o] = pd.read_csv("./CT/"+ o + ".csv")
    Max_freq[o] = np.zeros((Codon_Table[o].shape[0],1))
    
    for c in range(Codon_Table[o].shape[0]):
        subset = Codon_Table[o].loc[Codon_Table[o].iloc[:, 0]==Codon_Table[o].iloc[c, 0], :]
        
        Max_freq[o][c,0:1] = subset['relative_frequency'].max()
    
    Codon_Table[o] = pd.concat((Codon_Table[o], pd.DataFrame(Max_freq[o][:,0:1])), axis = 1)
    Codon_Table[o].columns = ['AA', 'codon', 'relative_frequency', 'Max_frequency']

In [7]:
Test_Data = pd.read_csv("./CodonTransformerPpr_Data/52proteinArch1.csv")

In [10]:
molecule = list(Test_Data.iloc[:,0])
Algorithms = ['Arch1']

In [19]:
Prop_ts= {} 
o = 'Pichia'

Prop_ts = np.zeros((len(molecule), 3))
for m in range(len(molecule)):
    seq = Test_Data.iloc[m,1]
    seq = seq.replace('T','U')
    Prop_ts[m, 0] = (seq.count('G') + seq.count('C'))/len(seq)

    for k in np.arange(0,len(seq),3):
        temp = Codon_Table[o].loc[Codon_Table[o].iloc[:, 1]==seq[k:k+3], :]
        if temp['relative_frequency'].values <0.3:
            Prop_ts[m,1] = Prop_ts[m,1] + 1 #rare codons 

        Prop_ts[m,2] = Prop_ts[m,2] + np.log(temp['relative_frequency'].values/temp['Max_frequency'].values)

    Prop_ts[m,1] = Prop_ts[m,1]/(len(seq)/3)
    Prop_ts[m,2] = np.exp(Prop_ts[m,2]/(len(seq)/3))

    
df = pd.concat((Test_Data.iloc[:,0:1], pd.DataFrame(Prop_ts)), axis = 1)
df.columns = ['Molecules', 'GC','Rare','CSI']
df.to_excel("52protein_"+o+".xlsx")

In [15]:
np.arange(0,len(seq),3)

array([  0,   3,   6,   9,  12,  15,  18,  21,  24,  27,  30,  33,  36,
        39,  42,  45,  48,  51,  54,  57,  60,  63,  66,  69,  72,  75,
        78,  81,  84,  87,  90,  93,  96,  99, 102, 105, 108, 111, 114,
       117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147, 150, 153,
       156, 159, 162, 165, 168, 171, 174, 177, 180, 183, 186, 189, 192,
       195, 198, 201, 204, 207, 210, 213, 216, 219, 222, 225, 228, 231,
       234, 237, 240, 243, 246, 249, 252, 255, 258, 261, 264, 267, 270,
       273, 276, 279, 282, 285, 288, 291, 294, 297, 300, 303, 306, 309,
       312, 315, 318, 321, 324, 327, 330])